<h1>Task 2 Knodle Model</h2>

In [1]:
import knodle
import torch
import joblib
from transformers import AutoModelForSequenceClassification, AdamW
from knodle.trainer import AutoTrainer, AutoConfig, baseline, MajorityVoteTrainer, multi_trainer
import numpy as np
from knodle.trainer.baseline.config import MajorityConfig

2023-01-23 15:12:55,886 root         INFO     Initalized logger
/home/david/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-23 15:12:58.614129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 15:12:59.184247: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-23 15:12:59.292419: 

In [2]:
x_train = joblib.load('./data/train/x_train_sample.lib')
t_train = joblib.load('./data/train/t_matrix_sample.lib')
z_train = joblib.load('./data/train/z_matrix_sample.lib')
x_test = joblib.load('./data/test/x_test.lib')
y_test = joblib.load('./data/test/y_test.lib')

In [13]:
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer_type = "majority"

custom_model_config = AutoConfig.create_config(
    name=trainer_type,
    optimizer=AdamW,
    lr=1e-4,
    batch_size=16,
    epochs=2,
    filter_non_labelled=True,
)

print(custom_model_config.__dict__)

trainer = AutoTrainer(
    name="majority",
    model=model,
    mapping_rules_labels_t=np.array(t_train.to_dense()),
    model_input_x=x_train,
    rule_matches_z=np.array(z_train.to_dense()),
    trainer_config=custom_model_config,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

{'seed': None, 'caching_suffix': '', 'caching_folder': '/home/david/GitHub/Weak-Supervision-in-the-Medical-Domain/Task 2/cache', 'saved_models_dir': '/home/david/GitHub/Weak-Supervision-in-the-Medical-Domain/Task 2/cache', 'criterion': <function cross_entropy_with_probs at 0x7f1499077cb0>, 'lr': 0.0001, 'batch_size': 16, 'output_classes': 2, 'grad_clipping': None, 'device': device(type='cuda'), 'epochs': 2, 'optimizer': <class 'transformers.optimization.AdamW'>, 'class_weights': tensor([1., 1.]), 'filter_non_labelled': True, 'other_class_id': None, 'evaluate_with_other_class': False, 'ids2labels': None, 'max_rules': None, 'min_coverage': None, 'drop_rules': False, 'use_probabilistic_labels': True, 'probability_threshold': None}


2023-01-23 13:40:20,291 knodle.transformation.rule_reduction INFO     No filtering criteria ('max_rule' or 'min_coverage' for rule specified, returning the original rule matches.
/home/david/.local/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/home/david/.local/lib/python3.7/site-packages/knodle/transformation/majority.py:78: RuntimeWarning: invalid value encountered in true_divide
  rule_counts_probs = rule_counts / rule_counts.sum(axis=1).reshape(-1, 1)
2023-01-23 13:40:21,191 knodle.trainer.trainer INFO     ======================================
2023-01-23 13:40:21,193 knodle.trainer.trainer INFO     Training starts
2023-01-23 13:40:21,195 knodle.trainer.trainer INFO     ======================================
2023-01-23 13:40:29,406 kn

In [14]:
eval_dict, _ = trainer.test(x_test, y_test)
print(f"Accuracy: {eval_dict.get('accuracy')}")

100%|██████████| 491/491 [00:21<00:00, 22.89it/s]


Accuracy: 0.0


/home/david/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/david/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
